In [45]:
import tensorflow as tf

In [46]:
import numpy as np
import cv2

In [47]:
import glob
from os.path import join, basename

In [48]:
# batch_size = 256

In [49]:
def conv_net(data, variable_scope, reuse, name_appendix=None,
            data_format='channels_first'):
    with tf.variable_scope(variable_scope, reuse=tf.AUTO_REUSE):
        conv1 = tf.layers.conv2d(
            image_left, filters=96, kernel_size=11,
            strides=4, padding='VALID', activation=tf.nn.relu,
            data_format=data_format, name='conv1')
        conv1 = tf.layers.max_pooling2d(conv1, pool_size=3, strides=2,
                                        padding='VALID',
                                        data_format=data_format)
        conv1 = tf.nn.local_response_normalization(conv1, depth_radius=5, bias=1,
                                                  alpha=0.0001, beta=0.75)
        conv2 = tf.layers.conv2d(conv1, filters=256, kernel_size=5,
                                strides=1, padding='SAME', activation=tf.nn.relu,
                                data_format=data_format, name='conv2')
        conv2 = tf.layers.max_pooling2d(conv2, pool_size=3, strides=2,
                                       padding='VALID', data_format=data_format)
        conv2 = tf.nn.local_response_normalization(conv2, depth_radius=5,
                                                  bias=1, alpha=0.0001, beta=0.75)
        conv3 = tf.layers.conv2d(conv2, filters=384, kernel_size=3,
                                strides=1, padding='SAME', activation=tf.nn.relu,
                                data_format=data_format, name='conv3')
        conv4_name = 'conv4'
        if name_appendix is not None:
            conv4_name += '_' + name_appendix
        conv4 = tf.layers.conv2d(conv3, filters=64, kernel_size=1,
                            strides=1, padding='VALID', activation=tf.nn.relu,
                            data_format=data_format, name=conv4_name)
        conv4 = tf.contrib.layers.flatten(conv4)
    return conv4

In [50]:
def facegrid_net(data):
    flatten = tf.contrib.layers.flatten(data)
    fg_fc1 = tf.layers.dense(
        flatten, 256, activation=tf.nn.relu,                            
        kernel_initializer=tf.truncated_normal_initializer(mean=0, stddev=0.005),
        bias_initializer=tf.constant_initializer(1),
        name='fg_fc1'
    )
    fg_fc2 = tf.layers.dense(fg_fc1, 128, activation=tf.nn.relu,
                            name='fg_fc2')
    return fg_fc2

In [51]:
channels_last_img_size = (None, 224, 224, 3)
channels_last_grid_size = (None, 1, 1, 625)
channels_first_img_size = (None, 3, 224, 224)
channels_first_grid_size = (None, 625, 1, 1)

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    # Add multiple inputs
    image_left = tf.placeholder(tf.float32, (None, 224, 224, 3), name='image_left') # Channel first or last?
    image_right = tf.placeholder(tf.float32, (None, 224, 224, 3), name='image_right')
    image_face = tf.placeholder(tf.float32, (None, 224, 224, 3), name='image_face')
    facegrid = tf.placeholder(tf.float32, (None, 1, 1, 625), name='facegrid')
    
    # Eyes
    conv4_left = conv_net(image_left, 'eye_conv_net', reuse=False,
                         name_appendix='l')
    conv4_right = conv_net(image_right, 'eye_conv_net', reuse=True,
                          name_appendix='r')
    concat1 = tf.concat([conv4_left, conv4_right], 1)
    fc1 = tf.layers.dense(concat1, 128, activation=tf.nn.relu,
                         name='fc1')
    
    # Face
    conv4_face = conv_net(image_face, 'face_conv_net', reuse=None,
                         name_appendix=None)
    # Flatten first?
    with tf.variable_scope('face_conv_net'):
        fc1_f = tf.layers.dense(conv4_face, 128, activation=tf.nn.relu,
                                name='fc1')
        fc2_f = tf.layers.dense(fc1_f, 64, activation=tf.nn.relu,
                                name='fc2')
    
    # FaceGrid
    fg_fc2 = facegrid_net(facegrid)
    
    concat2 = tf.concat([fc1, fg_fc2, fc2_f], axis=1)
    fc2 = tf.layers.dense(concat2, 128, activation=tf.nn.relu,
                         name='fc2')
    fc3 = tf.layers.dense(fc2, 2, name='fc3')

ValueError: Negative dimension size caused by subtracting 11 from 3 for 'eye_conv_net/conv1/Conv2D' (op: 'Conv2D') with input shapes: [?,224,224,3], [11,11,224,96].

In [38]:
section_name_map = {'l': 'eye_conv_net',
                    'r': 'eye_conv_net',
                    'f': 'face_conv_net'}

def load_external_variables(directory_path):
    # conv{x}_{y} -> different mapping
    # read all file names
    # return {'new_variable_name': variable_value}
    file_names = glob.glob(join(directory_path,'*.npy'))
    name_2_var_map = {}
    for file_name in file_names:
        variable_name = basename(file_name).split('.')[0]
        variable_value = np.load(file_name)
        new_name = format_variable_name(variable_name)
        name_2_var_map[new_name] = variable_value
    return name_2_var_map
    
def format_variable_name(variable_name):
    name_parts = variable_name.strip().split('_')
    if int(name_parts.pop()) == 0:
        variable_type = 'kernel'
    else:
        variable_type = 'bias'
    if len(name_parts) == 2:
        # section name could be 'f', 'l', 'r', 'fg' 
        # with 'fg', the order is reversed
        layer_type, section_name = name_parts
        if layer_type == 'fg':
            layer_type = '_'.join(name_parts)
        variable_scope = section_name_map.get(section_name)
        if variable_scope is not None:
            # Special case for conv4, need conv4_l and conv4_r
            # This is too complicated, what about the other way around? 
            # Loading variable and check if numpy has it?
            if section_name in ('l', 'r') and layer_type == 'conv4': 
                layer_type += '_' + section_name
            new_name = '{}/{}/{}:0'.format(variable_scope,
                                          layer_type,
                                          variable_type)
        else:
            new_name = '{}/{}:0'.format(layer_type,
                                        variable_type)
    else:
        new_name = '{}/{}:0'.format(name_parts[0],
                                   variable_type)
    return new_name

In [39]:
def load_trained_variable(model_path, session):
    varname_2_value = load_external_variables(model_path)
    for var in tf.global_variables():
        try:
            session.run(var.assign(varname_2_value[var.name]))
        except KeyError:
            raise ValueError('Missing variable {}'.format(var.name))
        except ValueError as e:
            print('Error with variable {}'.format(var.name))
            raise e

In [40]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    load_trained_variable('caffe_models/weights_itracker25x_iter_92000_new',
                          sess)

Error with variable eye_conv_net/conv1/kernel:0


ValueError: Dimension 0 in both shapes must be equal, but are 11 and 96 for 'Assign' (op: 'Assign') with input shapes: [11,11,224,96], [96,3,11,11].

In [ ]:
tf.get_variable('')

In [ ]:
def predict():
    pass